In [4]:
import pandas as pd 
import xml.etree.ElementTree as ET
import numpy as np
import utils
from gym import spaces
import numpy as np

xmlfile = r"C:\Users\kilia\MASTER\V2Z1551.pml"

tree = ET.parse(xmlfile)
root = tree.getroot()
FEATURES = ["H", "HBA", "HBD", "exclusion"]

In [8]:
def set_tol(tree, id, newval, target=None):
    elm = tree.find(".//*[@featureId='"+str(id)+"']")
    if (elm.get("name") == "H") or (elm.get("type") == "exclusion"):
        elm.find("./position").set('tolerance', newval)
        return tree
    if target == "target":
        child = elm.find("./target")
        child.set('tolerance', newval)
        return tree
    if target == "origin":
        child = elm.find("./origin")
        child.set('origin', newval)
        return tree
    return False

def set_weight(id, newval):
    elm = tree.find(".//*[@featureId='"+str(id)+"']")
    elm.set("weight", newval)
    return tree

In [61]:
set_tol(tree=tree, id="837634895065800895", newval="10").write(r"C:\Users\kilia\MASTER\V2Z1551_modified.pml")

# EXPERIMENTAL

In [29]:
n_feats = np.array([[5,4,3,8], [3,3,3]], dtype=object)
nvec = []
# codec -> 0: H
#          1: HBA
#          2: HBD
#          3: EXCLUSION
#len(n_feats)
for i in range(len(n_feats)):
    nvec.append([])
    for j in range(len(n_feats[i])):
        nvec[i].append([])
        for k in range(n_feats[i][j]):
            nvec[i][j].append(4)
print(nvec)

[[[4, 4, 4, 4, 4], [4, 4, 4, 4], [4, 4, 4], [4, 4, 4, 4, 4, 4, 4, 4]], [[4, 4, 4], [4, 4, 4], [4, 4, 4]]]


In [10]:
n_feats = np.array([[5,4,3,8], [3,3,3,3]], dtype=object)

# codec -> 0: H
#          1: HBA
#          2: HBD
#          3: EXCLUSION


action_space = spaces.MultiDiscrete(n_feats)

observation_space = spaces.MultiDiscrete(n_feats)
observation_space.sample()

array([[3, 2, 2, 4],
       [1, 1, 2, 2]], dtype=int64)

In [5]:
featureIds = []
root = tree.getroot()
for i in range(len(FEATURES)):
    featureIds.append([])
    if FEATURES[i] != "exclusion":
        for elm in tree.findall(".//*[@name='"+FEATURES[i]+"']"):
            featureIds[i].append(elm.get("featureId"))
    else:
        for elm in tree.findall(".//*[@type='"+FEATURES[i]+"']"):
            featureIds[i].append(elm.get("featureId"))
print(featureIds)

[['83763489212130077', '837634895065800895'], ['837634894702000876', '837634894751500878', '837634894566000874'], ['837634894919400880'], ['837634894923500881', '837634892436200172', '8376348953118001029', '837634894708900877', '837634894581900875', '837634894754900879', '837634892449700179', '83763489201030076', '837634895061000894', '837634892440900173']]


In [6]:
# featureIds = []
# for i in range(len(FEATURES)):
#     featureIds.append([])
#     if FEATURES[i] != "exclusion":    
#         for elm in tree.findall(".//*[@name='"+FEATURES[i]+"']"):
#             featureIds[i].append(elm.get("featureId"))
#     else:
#         for elm in tree.findall(".//*[@type='"+FEATURES[i]+"']"):
#             featureIds[i].append(elm.get("featureId"))
def get_tol_and_weight(tree, id):
    """
    Get tolerance and weight of a feature
    :param tree: tree of Phar file
    :param id: featureId
    :return: tolerance and weight as separate values, when dealing with HBA and HBD, tolerance of target and origin as well as weight are returned
    """
    
    elm = tree.find(".//*[@featureId='"+id+"']")
    if (elm.get("name") == "H") or (elm.get("type") == "exclusion"):
        child = elm.find("./position")
        return child.get("tolerance"), elm.get("weight")
    else:
        child_target = elm.find("./target")
        child_origin = elm.find("./origin")
        return child_target.get('tolerance'), child_origin.get('tolerance'), elm.get('weight')



nvec = []
for feature in featureIds:
    nvec.append(len(feature))
    
# currently for one pharmacophore at a time
observation_space = []
for i in range(len(featureIds)):
    observation_space.append([])
    for id in featureIds[i]:
        observation_space[i].extend(get_tol_and_weight(tree, id))
        # for k in range(nfeats[i][j]):
        #     nvec[i][j].append()

ovec = []
for feature in observation_space:
    ovec.append(len(feature))

# action_space = spaces.MultiDiscrete(nvec*4)
# observation_space = spaces.MultiDiscrete(observation_space)

In [9]:
action_space = spaces.MultiDiscrete(np.array(nvec)*4)
print(action_space)

MultiDiscrete([ 8 12  4 40])


In [10]:
print(observation_space)
observation_space = spaces.Discrete(sum(ovec))
print(observation_space)


[['1.5', '1.0', '1.5', '1.0'], ['1.5', '1.9499999', '1.0', '1.5', '1.9499999', '1.0', '1.5', '1.9499999', '1.0'], ['1.9499999', '1.5', '1.0'], ['1.0', '1.0', '1.5', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.5', '1.0', '1.5', '1.0', '1.5', '1.0', '1.5', '1.0']]
Discrete(36)


In [4]:
11%4
20//4

5

In [9]:
print(featureIds)

[['83763489212130077', '837634895065800895'], ['837634894702000876', '837634894751500878', '837634894566000874'], ['837634894919400880'], ['837634894923500881', '837634892436200172', '8376348953118001029', '837634894708900877', '837634894581900875', '837634894754900879', '837634892449700179', '83763489201030076', '837634895061000894', '837634892440900173']]


In [11]:
onvec = []
for i in range(len(featureIds)):
    s = 0
    for id in featureIds[i]:
        s += len(get_tol_and_weight(tree=tree, id=id))
        
    onvec.append(s)

In [58]:
def get_observation_space():
    observation_space = np.array([])
    for i in range(len(featureIds)):
        x = np.array([], dtype=np.float32)
        for id in featureIds[i]:
            x = np.append(x, utils.get_tol_and_weight(tree, id))
        observation_space = np.append(observation_space, x)
    return observation_space

bounds = {"H": [1, 4], "HBA": [1, 5], "HBD": [1, 5], "exclusion": [0, 10], "WGHT": [0.1, 3]}

def get_initial_os():
    wght_low = bounds["WGHT"][0]
    wght_up = bounds["WGHT"][1]
    d = bounds.copy()
    d.popitem()
    for i in range(len(bounds.keys())-1):
        feature = list(bounds.keys())[i]
        lower = bounds[feature][0]
        upper = bounds[feature][1]
        up = []
        down = []
        if feature == "exclusion" or feature == "H":   
            for _ in featureIds[i]:
                up.extend([upper, wght_up])
                down.extend([lower, wght_low])
            d[feature] = spaces.Box(low=np.array(down), high=np.array(up), shape=(len(featureIds[i])*2,), dtype=np.float32)
        if feature == "HBA" or feature == "HBD":
            for _ in featureIds[i]:
                up.extend([upper, upper, wght_up])
                down.extend([lower, lower, wght_low])
            d[feature] = spaces.Box(low=np.array(down), high=np.array(up), shape=(len(featureIds[i])*3,), dtype=np.float32)
    return d

In [60]:
os = spaces.Dict(get_initial_os())
os.sample()

(4,)
(20,)


c:\Users\kilia\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


OrderedDict([('H',
              array([3.7234685, 1.5904604, 2.2449753, 2.0391967], dtype=float32)),
             ('HBA',
              array([3.6233723 , 4.7737703 , 2.0500684 , 1.4501897 , 3.2240496 ,
                     0.62952703, 4.395515  , 2.034323  , 1.9348621 ], dtype=float32)),
             ('HBD', array([3.553263 , 1.2815524, 2.5145724], dtype=float32)),
             ('exclusion',
              array([4.5976105 , 1.9447223 , 5.6468463 , 1.9731505 , 6.4719625 ,
                     0.7851936 , 1.4741234 , 0.12440996, 8.091228  , 1.8296797 ,
                     0.3141109 , 2.5040352 , 5.8465414 , 1.1841367 , 9.899724  ,
                     1.3313555 , 5.2602425 , 0.18746142, 5.05088   , 2.798076  ],
                    dtype=float32))])

In [12]:
def get_observation():
        os_space = dict()
        for i, f in zip(range(len(featureIds)),FEATURES):
            x = []
            for id in featureIds[i]:
                x.extend(utils.get_tol_and_weight(tree, id))
            os_space[f] = np.array(x, dtype=np.float32)
        return os_space

In [13]:
get_observation()

{'H': array([1.5, 1. , 1.5, 1. ], dtype=float32),
 'HBA': array([1.5      , 1.9499999, 1.       , 1.5      , 1.9499999, 1.       ,
        1.5      , 1.9499999, 1.       ], dtype=float32),
 'HBD': array([1.9499999, 1.5      , 1.       ], dtype=float32),
 'exclusion': array([1. , 1. , 1.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1.5,
        1. , 1.5, 1. , 1.5, 1. , 1.5, 1. ], dtype=float32)}

In [12]:
import numpy as np
bounds = {"H": [1, 4], "HBA": [1, 5], "HBD": [1, 5]}
last_observation = {'H': np.array([1.5, 1.7, 1.5, 1.1]),
                    'HBA': np.array([1.5, 1.9499999, 1., 1.5, 1.9499999, 1., 1.5, 1.9499999, 1.]),
                    'HBD': np.array([1.9499999, 2, 1.1])}
terminated = False
for key in last_observation.keys():
            if terminated == True:
                break
            terminated = not(np.all(np.logical_and(last_observation[key] >= bounds[key][0], last_observation[key] <= bounds[key][1])))
terminated

False